In [13]:
# importa as funções de captura de movimento (motion capture - mocap)
from mocap import *
# importa o pacote para gestão de diretorios
import os

# Configuração da analise

In [24]:
# defina um identificar para os dados. Sugiro definir nome do sujeito e tipo de teste
id_sujeito = 'suj02_botcae'
# define o caminho ate o video
vid_path = "./dados/BOTECAE_T3.MP4"
# define o tamanho do objeto conhecido para calibração
known_distance = 0.50
# define taxa de quadros do video
fps = 120
# defive se o video esta invertido verticalmente
fliped = False

# Procedimentos manuais

In [22]:
# pressione espaço para marcar o inicio e novamente para marcar o fim do periodo de análise
# busque definir o periodo de analise entre o momento em que o sujeito toca definitivamente o suporte para o salto
# ate o momento em que ele solta o suporte superior após realizar o salto
# veja o video mais de uma vez se necessário
# aperte Q para sair do video
# caso algum frame apresente problemas, sera indicado junto a frase (quebra de exbição), não há ação relacionada a essa observação
periodo_analise = get_analysis_epoch(vid_path,fliped=fliped)

Quebra de exibição. frame  102
começo: 1740
fim: 2616


In [23]:
# selecione a area para análise, clique e arraste para selecionar a area do video de forma que apenas fique visivel o sujeito a ser analisado
# apos a seleção, aperte qualquer espaço
roi = get_roi(vid_path,periodo_analise,fliped=fliped)

In [25]:
# calibre a distancia medida por uma distancia conhecida no video, para isso:
# clique e segure no inicio da barra horiontal em que o sujeito se apoia ao final do bote e arraste ate o final dessa mesma barra
# apos, presisone a tecla Q
distance_pixels = Calibrator().get_distance_pixels(vid_path,periodo_analise,fliped=fliped)
calib_factor = known_distance/distance_pixels

# Estimação de pose - automatizado

In [26]:
# rode essa celula para analizar o video - aguarde a janela fechar automaticamente
# essa função extrai os dados de cinematica projetada no 2D
coords = analyze_video(vid_path,periodo_analise,roi,fliped=fliped)
# essa função filtra os sinais com um passa baixa e calibra as distancias
coords = process_coords(coords,calib_factor)
# calcula os angulos projetados no 2D com os dados obtidos
joint_angles = get_joint_angles(articulacoes,coords)

# Exibicao

In [10]:
# apresenta graficamente os angulos 2D
%matplotlib qt
show_angular_kinematics(coords,roi,fps=120)

(<Figure size 800x500 with 5 Axes>,
 <matplotlib.animation.FuncAnimation at 0x24c86419e50>)

In [12]:
# apresenta algumas variaveis cinematicas
show_linear_kinematics(coords,roi,fps=120)

(<Figure size 800x500 with 4 Axes>,
 <matplotlib.animation.FuncAnimation at 0x24c8925e4f0>)

In [17]:
# salva as informações extraidas com o nome indicado na configuração
# tenta criar um diretorio para salvar o arquivo
try:
    os.makedirs(os.path.join('./resultados',id_sujeito))
except:
    print("Dado já analisado anteriormente")
# salva o arquivo
save_excel(os.path.join('./resultados',id_sujeito,id_sujeito+'.xlsx'),coords,joint_angles)